# Advanced Copper Market Analysis

Multi-dimensional analysis of copper futures market structure, patterns, and dynamics.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings

# Add project root to Python path
project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)

from config.database_config import get_connection_string

warnings.filterwarnings('ignore')

# Enhanced style settings
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['figure.dpi'] = 120
plt.rcParams['savefig.dpi'] = 200

# Professional color palette
COLORS = {
    'primary': '#1f77b4',
    'secondary': '#ff7f0e', 
    'tertiary': '#2ca02c',
    'quaternary': '#d62728',
    'quinary': '#9467bd',
    'dark': '#2c3e50',
    'light': '#ecf0f1'
}

# Gradient palettes for different analysis types
risk_colors = ['#27ae60', '#f39c12', '#e74c3c']  # Green -> Orange -> Red
time_colors = ['#3498db', '#9b59b6', '#e91e63', '#ff5722']  # Blue -> Purple -> Pink -> Orange
performance_colors = ['#2ecc71', '#f1c40f', '#e67e22', '#c0392b']  # Green -> Yellow -> Orange -> Red

print("Advanced analysis libraries loaded successfully")

## Data Loading and Preprocessing

In [ ]:
def load_comprehensive_data(conn, days=365):
    """Load comprehensive futures data for analysis"""
    query = f"""
    SELECT 
        p.TradeDate,
        m.MetalCode,
        t.TenorTypeName,
        p.SettlementPrice,
        p.Volume,
        p.OpenInterest,
        CASE 
            WHEN t.TenorTypeName LIKE 'Generic 1st%' THEN 1
            WHEN t.TenorTypeName LIKE 'Generic 2nd%' THEN 2
            WHEN t.TenorTypeName LIKE 'Generic 3rd%' THEN 3
            WHEN t.TenorTypeName LIKE 'Generic 4th%' THEN 4
            WHEN t.TenorTypeName LIKE 'Generic 5th%' THEN 5
            WHEN t.TenorTypeName LIKE 'Generic 6th%' THEN 6
            WHEN t.TenorTypeName LIKE 'Generic 7th%' THEN 7
            WHEN t.TenorTypeName LIKE 'Generic 8th%' THEN 8
            WHEN t.TenorTypeName LIKE 'Generic 9th%' THEN 9
            WHEN t.TenorTypeName LIKE 'Generic 10th%' THEN 10
            WHEN t.TenorTypeName LIKE 'Generic 11th%' THEN 11
            WHEN t.TenorTypeName LIKE 'Generic 12th%' THEN 12
            WHEN t.TenorTypeName = 'Cash' THEN 0
            WHEN t.TenorTypeName = 'Tom-Next' THEN 0.1
            WHEN t.TenorTypeName = '3M Futures' THEN 3
            ELSE NULL
        END as TenorNumber
    FROM T_CommodityPrice p
    INNER JOIN M_Metal m ON p.MetalID = m.MetalID
    INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
    WHERE 
        m.MetalCode = 'COPPER'
        AND p.TradeDate >= DATEADD(day, -{days}, GETDATE())
        AND p.SettlementPrice IS NOT NULL
    ORDER BY p.TradeDate DESC, t.TenorTypeName
    """
    
    df = pd.read_sql(query, conn)
    df['TradeDate'] = pd.to_datetime(df['TradeDate'])
    df = df.dropna(subset=['TenorNumber'])
    return df

# Connect and load data
conn = pyodbc.connect(get_connection_string())
print("Connected to database")

# Load 1 year of data for comprehensive analysis
df = load_comprehensive_data(conn, days=365)
print(f"Loaded {len(df):,} records covering {df['TradeDate'].nunique()} trading days")
print(f"Date range: {df['TradeDate'].min().date()} to {df['TradeDate'].max().date()}")
print(f"Tenors available: {sorted(df['TenorNumber'].unique())}")

# Create pivot tables for different analyses
price_pivot = df.pivot_table(
    values='SettlementPrice', index='TradeDate', columns='TenorNumber', aggfunc='mean'
).sort_index()

volume_pivot = df.pivot_table(
    values='Volume', index='TradeDate', columns='TenorNumber', aggfunc='mean'
).sort_index()

oi_pivot = df.pivot_table(
    values='OpenInterest', index='TradeDate', columns='TenorNumber', aggfunc='mean'
).sort_index()

print(f"\nPivot tables created: {price_pivot.shape[0]} dates x {price_pivot.shape[1]} tenors")

## 1. Term Structure Evolution Analysis

In [ ]:
# Calculate term structure metrics over time
def calculate_term_structure_metrics(price_data):
    """Calculate various term structure metrics"""
    metrics = pd.DataFrame(index=price_data.index)
    
    # Basic spreads
    if 1 in price_data.columns and 3 in price_data.columns:
        metrics['M1_M3_Spread'] = price_data[3] - price_data[1]
        metrics['M1_M3_Ratio'] = price_data[3] / price_data[1]
    
    if 1 in price_data.columns and 6 in price_data.columns:
        metrics['M1_M6_Spread'] = price_data[6] - price_data[1]
        
    if 1 in price_data.columns and 12 in price_data.columns:
        metrics['M1_M12_Spread'] = price_data[12] - price_data[1]
    
    # Curve steepness (slope between front and back)
    front_tenors = [col for col in price_data.columns if col <= 3]
    back_tenors = [col for col in price_data.columns if col >= 9]
    
    if front_tenors and back_tenors:
        front_avg = price_data[front_tenors].mean(axis=1)
        back_avg = price_data[back_tenors].mean(axis=1)
        metrics['Curve_Steepness'] = back_avg - front_avg
    
    # Curve convexity (butterfly spreads)
    if 1 in price_data.columns and 3 in price_data.columns and 6 in price_data.columns:
        metrics['Butterfly_1_3_6'] = price_data[3] - (price_data[1] + price_data[6]) / 2
    
    return metrics.dropna(how='all')

# Calculate metrics
ts_metrics = calculate_term_structure_metrics(price_pivot)

# Create comprehensive term structure visualization
fig = plt.figure(figsize=(18, 12))
gs = GridSpec(3, 2, figure=fig, hspace=0.3, wspace=0.3)

# 1. Term structure evolution heatmap
ax1 = fig.add_subplot(gs[0, :])
# Calculate relative price changes
price_changes = price_pivot.pct_change(periods=5).rolling(window=5).mean() * 100
sns.heatmap(price_changes.T, cmap='RdBu_r', center=0, 
           ax=ax1, cbar_kws={'label': '5-day Return (%)'}, 
           xticklabels=20, yticklabels=True)
ax1.set_title('Term Structure Evolution: 5-Day Rolling Returns', fontsize=14, weight='bold')
ax1.set_xlabel('Trading Date')
ax1.set_ylabel('Contract Month')

# 2. Key spreads over time
ax2 = fig.add_subplot(gs[1, 0])
if 'M1_M3_Spread' in ts_metrics.columns:
    ax2.plot(ts_metrics.index, ts_metrics['M1_M3_Spread'], 
            color=COLORS['primary'], linewidth=2, label='M1-M3')
if 'M1_M6_Spread' in ts_metrics.columns:
    ax2.plot(ts_metrics.index, ts_metrics['M1_M6_Spread'], 
            color=COLORS['secondary'], linewidth=2, label='M1-M6')
if 'M1_M12_Spread' in ts_metrics.columns:
    ax2.plot(ts_metrics.index, ts_metrics['M1_M12_Spread'], 
            color=COLORS['tertiary'], linewidth=2, label='M1-M12')

ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax2.set_title('Calendar Spreads Evolution', fontsize=12, weight='bold')
ax2.set_ylabel('Spread (USD/tonne)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Curve steepness and convexity
ax3 = fig.add_subplot(gs[1, 1])
if 'Curve_Steepness' in ts_metrics.columns:
    ax3.plot(ts_metrics.index, ts_metrics['Curve_Steepness'], 
            color=COLORS['quaternary'], linewidth=2, label='Steepness')
if 'Butterfly_1_3_6' in ts_metrics.columns:
    ax3_twin = ax3.twinx()
    ax3_twin.plot(ts_metrics.index, ts_metrics['Butterfly_1_3_6'], 
                 color=COLORS['quinary'], linewidth=2, label='Butterfly 1-3-6')
    ax3_twin.set_ylabel('Butterfly (USD/tonne)', color=COLORS['quinary'])
    ax3_twin.legend(loc='upper right')

ax3.set_title('Curve Shape Dynamics', fontsize=12, weight='bold')
ax3.set_ylabel('Steepness (USD/tonne)', color=COLORS['quaternary'])
ax3.legend(loc='upper left')
ax3.grid(True, alpha=0.3)

# 4. Contango/Backwardation analysis
ax4 = fig.add_subplot(gs[2, :])
if 'M1_M3_Spread' in ts_metrics.columns:
    # Create contango/backwardation indicators
    contango_periods = ts_metrics['M1_M3_Spread'] > 0
    backwardation_periods = ts_metrics['M1_M3_Spread'] < 0
    
    ax4.fill_between(ts_metrics.index, 0, ts_metrics['M1_M3_Spread'], 
                    where=contango_periods, color='green', alpha=0.3, 
                    label='Contango', interpolate=True)
    ax4.fill_between(ts_metrics.index, 0, ts_metrics['M1_M3_Spread'], 
                    where=backwardation_periods, color='red', alpha=0.3, 
                    label='Backwardation', interpolate=True)
    
    ax4.plot(ts_metrics.index, ts_metrics['M1_M3_Spread'], 
            color='black', linewidth=1.5, alpha=0.8)
    
    # Calculate percentage of time in each state
    total_days = len(ts_metrics.dropna())
    contango_pct = (contango_periods.sum() / total_days) * 100
    backwardation_pct = (backwardation_periods.sum() / total_days) * 100
    
    ax4.text(0.02, 0.95, f'Contango: {contango_pct:.1f}% of time', 
            transform=ax4.transAxes, fontsize=10, 
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    ax4.text(0.02, 0.85, f'Backwardation: {backwardation_pct:.1f}% of time', 
            transform=ax4.transAxes, fontsize=10,
            bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))

ax4.axhline(y=0, color='black', linestyle='-', alpha=0.8)
ax4.set_title('Market Structure: Contango vs Backwardation (M1-M3)', fontsize=12, weight='bold')
ax4.set_xlabel('Trading Date')
ax4.set_ylabel('M1-M3 Spread (USD/tonne)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.suptitle('Copper Futures Term Structure Dynamics', fontsize=16, weight='bold', y=0.98)
plt.tight_layout()
plt.show()

# Summary statistics
print("\nTerm Structure Summary Statistics:")
print(ts_metrics.describe().round(2))

## 2. Volatility Surface and Risk Analysis

In [ ]:
# Calculate rolling volatilities for different periods
def calculate_volatility_metrics(price_data, windows=[5, 20, 60]):
    """Calculate volatility metrics for different time windows"""
    returns = price_data.pct_change().dropna()
    volatilities = {}
    
    for window in windows:
        vol_name = f'{window}d_vol'
        volatilities[vol_name] = returns.rolling(window=window).std() * np.sqrt(252) * 100
    
    return pd.concat(volatilities, axis=1)

# Calculate volatilities
vol_metrics = calculate_volatility_metrics(price_pivot)

# Create volatility analysis
fig = plt.figure(figsize=(18, 12))
gs = GridSpec(3, 3, figure=fig, hspace=0.4, wspace=0.3)

# 1. Volatility surface (heatmap)
ax1 = fig.add_subplot(gs[0, :])
# Use 20-day volatility for surface
vol_20d = vol_metrics['20d_vol'].dropna(how='all')
if not vol_20d.empty:
    sns.heatmap(vol_20d.T, cmap='plasma', 
               ax=ax1, cbar_kws={'label': 'Volatility (% annualized)'}, 
               xticklabels=20, yticklabels=True)
    ax1.set_title('Volatility Surface: 20-Day Rolling Volatility', fontsize=14, weight='bold')
    ax1.set_xlabel('Trading Date')
    ax1.set_ylabel('Contract Month')

# 2. Term structure of volatility (current)
ax2 = fig.add_subplot(gs[1, 0])
latest_vol = vol_20d.iloc[-1].dropna()
if not latest_vol.empty:
    bars = ax2.bar(range(len(latest_vol)), latest_vol.values, 
                  color=[risk_colors[min(i, len(risk_colors)-1)] for i in range(len(latest_vol))], 
                  alpha=0.8)
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, latest_vol.values)):
        ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                f'{val:.1f}%', ha='center', va='bottom', fontsize=9, weight='bold')
    
    ax2.set_title(f'Current Volatility by Tenor\n({latest_vol.name.strftime("%Y-%m-%d")})', 
                 fontsize=12, weight='bold')
    ax2.set_xlabel('Contract Month')
    ax2.set_ylabel('Volatility (%)')
    ax2.set_xticks(range(len(latest_vol)))
    ax2.set_xticklabels([f'M{int(idx)}' for idx in latest_vol.index])
    ax2.grid(True, alpha=0.3, axis='y')

# 3. Volatility time series for key contracts
ax3 = fig.add_subplot(gs[1, 1:])
key_tenors = [1, 3, 6, 12]
colors = [COLORS['primary'], COLORS['secondary'], COLORS['tertiary'], COLORS['quaternary']]

for i, tenor in enumerate(key_tenors):
    if tenor in vol_20d.columns:
        ax3.plot(vol_20d.index, vol_20d[tenor], 
                color=colors[i], linewidth=2, label=f'M{tenor}', alpha=0.8)

ax3.set_title('Volatility Evolution: Key Contracts', fontsize=12, weight='bold')
ax3.set_xlabel('Trading Date')
ax3.set_ylabel('20-Day Volatility (%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Volatility distribution analysis
ax4 = fig.add_subplot(gs[2, 0])
if 1 in vol_20d.columns:
    front_month_vol = vol_20d[1].dropna()
    ax4.hist(front_month_vol, bins=30, density=True, alpha=0.7, 
            color=COLORS['primary'], edgecolor='black')
    
    # Add distribution statistics
    mean_vol = front_month_vol.mean()
    std_vol = front_month_vol.std()
    ax4.axvline(mean_vol, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_vol:.1f}%')
    ax4.axvline(mean_vol + std_vol, color='orange', linestyle=':', linewidth=2, 
               label=f'+1σ: {mean_vol + std_vol:.1f}%')
    ax4.axvline(mean_vol - std_vol, color='orange', linestyle=':', linewidth=2, 
               label=f'-1σ: {mean_vol - std_vol:.1f}%')
    
    ax4.set_title('Front Month Volatility Distribution', fontsize=12, weight='bold')
    ax4.set_xlabel('Volatility (%)')
    ax4.set_ylabel('Density')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

# 5. Risk metrics table
ax5 = fig.add_subplot(gs[2, 1:])
ax5.axis('off')

# Calculate risk metrics
returns_1m = price_pivot[1].pct_change().dropna() * 100 if 1 in price_pivot.columns else None

if returns_1m is not None:
    risk_metrics = {
        'Current Volatility (20d)': f"{latest_vol[1]:.1f}%" if 1 in latest_vol.index else "N/A",
        'Average Volatility': f"{vol_20d[1].mean():.1f}%" if 1 in vol_20d.columns else "N/A",
        'Max Volatility (1Y)': f"{vol_20d[1].max():.1f}%" if 1 in vol_20d.columns else "N/A",
        'Min Volatility (1Y)': f"{vol_20d[1].min():.1f}%" if 1 in vol_20d.columns else "N/A",
        'Daily VaR (95%)': f"{np.percentile(returns_1m, 5):.2f}%",
        'Daily VaR (99%)': f"{np.percentile(returns_1m, 1):.2f}%",
        'Max Daily Loss (1Y)': f"{returns_1m.min():.2f}%",
        'Max Daily Gain (1Y)': f"{returns_1m.max():.2f}%",
        'Skewness': f"{stats.skew(returns_1m):.2f}",
        'Kurtosis': f"{stats.kurtosis(returns_1m):.2f}"
    }
    
    # Create table
    table_data = [[metric, value] for metric, value in risk_metrics.items()]
    table = ax5.table(cellText=table_data, 
                     colLabels=['Risk Metric', 'Value'],
                     cellLoc='left',
                     loc='center',
                     colWidths=[0.6, 0.4])
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 2)
    
    # Style the table
    for i in range(len(table_data) + 1):
        for j in range(2):
            cell = table[(i, j)]
            if i == 0:  # Header
                cell.set_facecolor('#40466e')
                cell.set_text_props(weight='bold', color='white')
            else:
                cell.set_facecolor('#f1f1f2' if i % 2 == 0 else 'white')
    
    ax5.set_title('Risk Metrics Summary (Front Month)', fontsize=12, weight='bold', pad=20)

plt.suptitle('Copper Futures Volatility and Risk Analysis', fontsize=16, weight='bold', y=0.98)
plt.tight_layout()
plt.show()

print(f"\nVolatility Analysis Complete. Current front month volatility: {latest_vol[1]:.1f}%" if 1 in latest_vol.index else "\nVolatility analysis complete.")

## 3. Liquidity and Market Microstructure Analysis

In [ ]:
# Advanced liquidity analysis
def calculate_liquidity_metrics(volume_data, oi_data, price_data):
    """Calculate comprehensive liquidity metrics"""
    metrics = pd.DataFrame(index=price_data.index)
    
    # Volume-based metrics
    metrics['Total_Volume'] = volume_data.sum(axis=1, skipna=True)
    metrics['Volume_Concentration'] = volume_data.apply(
        lambda x: (x.nlargest(3).sum() / x.sum()) if x.sum() > 0 else np.nan, axis=1
    )
    
    # Open Interest metrics
    metrics['Total_OI'] = oi_data.sum(axis=1, skipna=True)
    metrics['OI_Concentration'] = oi_data.apply(
        lambda x: (x.nlargest(3).sum() / x.sum()) if x.sum() > 0 else np.nan, axis=1
    )
    
    # Volume/OI ratio
    metrics['Volume_OI_Ratio'] = metrics['Total_Volume'] / metrics['Total_OI']
    
    # Price impact proxy (volume-weighted average price dispersion)
    if len(price_data.columns) > 1:
        weighted_prices = (price_data * volume_data).sum(axis=1) / volume_data.sum(axis=1)
        simple_avg_prices = price_data.mean(axis=1)
        metrics['Price_Dispersion'] = abs(weighted_prices - simple_avg_prices) / simple_avg_prices * 100
    
    return metrics.dropna(how='all')

# Calculate liquidity metrics
liquidity_metrics = calculate_liquidity_metrics(volume_pivot, oi_pivot, price_pivot)

# Create liquidity analysis visualization
fig = plt.figure(figsize=(18, 14))
gs = GridSpec(4, 3, figure=fig, hspace=0.4, wspace=0.3)

# 1. Volume concentration over time
ax1 = fig.add_subplot(gs[0, :])
# Plot volume by tenor as stacked area
volume_pivot_clean = volume_pivot.fillna(0)
tenor_columns = sorted([col for col in volume_pivot_clean.columns if col >= 1])

if tenor_columns:
    ax1.stackplot(volume_pivot_clean.index, 
                 *[volume_pivot_clean[col] for col in tenor_columns],
                 labels=[f'M{int(col)}' for col in tenor_columns],
                 colors=plt.cm.Set3(np.linspace(0, 1, len(tenor_columns))),
                 alpha=0.8)
    
    ax1.set_title('Volume Distribution Across Contract Months', fontsize=14, weight='bold')
    ax1.set_xlabel('Trading Date')
    ax1.set_ylabel('Daily Volume (contracts)')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)

# 2. Liquidity concentration metrics
ax2 = fig.add_subplot(gs[1, 0])
if 'Volume_Concentration' in liquidity_metrics.columns:
    ax2.plot(liquidity_metrics.index, liquidity_metrics['Volume_Concentration'] * 100, 
            color=COLORS['primary'], linewidth=2, label='Volume')
if 'OI_Concentration' in liquidity_metrics.columns:
    ax2.plot(liquidity_metrics.index, liquidity_metrics['OI_Concentration'] * 100, 
            color=COLORS['secondary'], linewidth=2, label='Open Interest')

ax2.set_title('Liquidity Concentration\n(Top 3 Contracts)', fontsize=12, weight='bold')
ax2.set_ylabel('Concentration (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Volume vs Open Interest dynamics
ax3 = fig.add_subplot(gs[1, 1])
if 'Volume_OI_Ratio' in liquidity_metrics.columns:
    ratio_clean = liquidity_metrics['Volume_OI_Ratio'].replace([np.inf, -np.inf], np.nan).dropna()
    ax3.plot(ratio_clean.index, ratio_clean, 
            color=COLORS['tertiary'], linewidth=2)
    
    # Add trend line
    z = np.polyfit(range(len(ratio_clean)), ratio_clean.values, 1)
    p = np.poly1d(z)
    ax3.plot(ratio_clean.index, p(range(len(ratio_clean))), 
            "--", color='red', alpha=0.8, linewidth=2, label='Trend')
    
    ax3.set_title('Volume/Open Interest Ratio', fontsize=12, weight='bold')
    ax3.set_ylabel('Ratio')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

# 4. Market efficiency proxy
ax4 = fig.add_subplot(gs[1, 2])
if 'Price_Dispersion' in liquidity_metrics.columns:
    dispersion_clean = liquidity_metrics['Price_Dispersion'].dropna()
    ax4.plot(dispersion_clean.index, dispersion_clean, 
            color=COLORS['quaternary'], linewidth=2)
    
    # Add efficiency zones
    mean_disp = dispersion_clean.mean()
    std_disp = dispersion_clean.std()
    
    ax4.axhline(mean_disp, color='green', linestyle='--', alpha=0.7, label='Average')
    ax4.axhline(mean_disp + std_disp, color='orange', linestyle=':', alpha=0.7, label='+1σ')
    ax4.fill_between(dispersion_clean.index, 0, mean_disp, alpha=0.2, color='green', label='Efficient')
    ax4.fill_between(dispersion_clean.index, mean_disp, mean_disp + std_disp, 
                    alpha=0.2, color='yellow', label='Moderate')
    
    ax4.set_title('Market Efficiency Proxy\n(Price Dispersion)', fontsize=12, weight='bold')
    ax4.set_ylabel('Dispersion (%)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

# 5. Tenor-specific liquidity heatmap
ax5 = fig.add_subplot(gs[2, :])
# Calculate normalized liquidity score
volume_norm = volume_pivot.div(volume_pivot.sum(axis=1), axis=0).fillna(0)
oi_norm = oi_pivot.div(oi_pivot.sum(axis=1), axis=0).fillna(0)
liquidity_score = (volume_norm + oi_norm) / 2

if not liquidity_score.empty:
    sns.heatmap(liquidity_score.T, cmap='YlOrRd', 
               ax=ax5, cbar_kws={'label': 'Normalized Liquidity Score'}, 
               xticklabels=20, yticklabels=True)
    ax5.set_title('Liquidity Distribution Heatmap (Volume + Open Interest)', fontsize=14, weight='bold')
    ax5.set_xlabel('Trading Date')
    ax5.set_ylabel('Contract Month')

# 6. Current market structure analysis
ax6 = fig.add_subplot(gs[3, :])

# Get latest data for market structure
latest_date = df['TradeDate'].max()
latest_market = df[df['TradeDate'] == latest_date].set_index('TenorNumber')

if not latest_market.empty:
    # Create subplot for different metrics
    tenors = sorted(latest_market.index)
    
    # Volume bars
    ax6_vol = ax6
    volume_bars = ax6_vol.bar([t - 0.2 for t in tenors], latest_market.loc[tenors, 'Volume'], 
                             width=0.4, label='Volume', color=COLORS['primary'], alpha=0.7)
    
    # Open Interest on secondary axis
    ax6_oi = ax6_vol.twinx()
    oi_bars = ax6_oi.bar([t + 0.2 for t in tenors], latest_market.loc[tenors, 'OpenInterest'], 
                        width=0.4, label='Open Interest', color=COLORS['secondary'], alpha=0.7)
    
    ax6_vol.set_xlabel('Contract Month')
    ax6_vol.set_ylabel('Volume', color=COLORS['primary'])
    ax6_oi.set_ylabel('Open Interest', color=COLORS['secondary'])
    ax6_vol.set_title(f'Current Market Structure - {latest_date.strftime("%Y-%m-%d")}', 
                     fontsize=14, weight='bold')
    
    ax6_vol.set_xticks(tenors)
    ax6_vol.set_xticklabels([f'M{int(t)}' for t in tenors])
    
    # Combined legend
    lines1, labels1 = ax6_vol.get_legend_handles_labels()
    lines2, labels2 = ax6_oi.get_legend_handles_labels()
    ax6_vol.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
    
    ax6_vol.grid(True, alpha=0.3)

plt.suptitle('Copper Futures Liquidity and Market Microstructure Analysis', 
             fontsize=16, weight='bold', y=0.98)
plt.tight_layout()
plt.show()

# Summary statistics
print("\nLiquidity Metrics Summary:")
if not liquidity_metrics.empty:
    print(liquidity_metrics.describe().round(3))
else:
    print("No liquidity metrics calculated.")

## 4. Principal Component Analysis of Term Structure

In [ ]:
# PCA analysis of term structure movements
def perform_pca_analysis(price_data, min_periods=100):
    """Perform PCA on term structure movements"""
    # Calculate returns
    returns = price_data.pct_change().dropna()
    
    # Filter out columns with insufficient data
    valid_columns = returns.columns[returns.count() >= min_periods]
    returns_clean = returns[valid_columns].dropna()
    
    if len(returns_clean) < min_periods or len(valid_columns) < 3:
        return None, None, None, None
    
    # Standardize the data
    scaler = StandardScaler()
    returns_scaled = scaler.fit_transform(returns_clean)
    
    # Perform PCA
    pca = PCA()
    pca_result = pca.fit_transform(returns_scaled)
    
    # Create component DataFrame
    components_df = pd.DataFrame(
        pca.components_.T, 
        columns=[f'PC{i+1}' for i in range(len(pca.components_))],
        index=valid_columns
    )
    
    # Create scores DataFrame
    scores_df = pd.DataFrame(
        pca_result,
        columns=[f'PC{i+1}' for i in range(pca_result.shape[1])],
        index=returns_clean.index
    )
    
    return pca, components_df, scores_df, pca.explained_variance_ratio_

# Perform PCA
pca_model, components, scores, explained_variance = perform_pca_analysis(price_pivot)

if pca_model is not None:
    # Create PCA visualization
    fig = plt.figure(figsize=(18, 12))
    gs = GridSpec(3, 3, figure=fig, hspace=0.4, wspace=0.3)
    
    # 1. Explained variance
    ax1 = fig.add_subplot(gs[0, 0])
    cumulative_var = np.cumsum(explained_variance)
    
    ax1.bar(range(1, len(explained_variance[:10]) + 1), explained_variance[:10] * 100, 
           alpha=0.7, color=COLORS['primary'], label='Individual')
    ax1.plot(range(1, len(cumulative_var[:10]) + 1), cumulative_var[:10] * 100, 
            'ro-', color=COLORS['secondary'], linewidth=2, label='Cumulative')
    
    ax1.set_title('PCA Explained Variance', fontsize=12, weight='bold')
    ax1.set_xlabel('Principal Component')
    ax1.set_ylabel('Explained Variance (%)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add variance labels
    for i, var in enumerate(explained_variance[:5]):
        ax1.text(i+1, var*100 + 1, f'{var*100:.1f}%', ha='center', fontsize=9, weight='bold')
    
    # 2. First three principal components
    ax2 = fig.add_subplot(gs[0, 1:])
    
    tenors = sorted(components.index)
    pc_colors = [COLORS['primary'], COLORS['secondary'], COLORS['tertiary']]
    
    for i, (pc, color) in enumerate(zip(['PC1', 'PC2', 'PC3'], pc_colors)):
        if pc in components.columns:
            ax2.plot(tenors, components.loc[tenors, pc], 
                    'o-', color=color, linewidth=2, markersize=6, 
                    label=f'{pc} ({explained_variance[i]*100:.1f}%)')
    
    ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax2.set_title('Principal Component Loadings', fontsize=12, weight='bold')
    ax2.set_xlabel('Contract Month')
    ax2.set_ylabel('Loading')
    ax2.set_xticks(tenors)
    ax2.set_xticklabels([f'M{int(t)}' for t in tenors])
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. PC1 vs PC2 scatter (recent period)
    ax3 = fig.add_subplot(gs[1, 0])
    recent_scores = scores.tail(252)  # Last year
    
    scatter = ax3.scatter(recent_scores['PC1'], recent_scores['PC2'], 
                         c=range(len(recent_scores)), cmap='viridis', 
                         alpha=0.6, s=20)
    
    ax3.set_title('PC1 vs PC2 (Last Year)', fontsize=12, weight='bold')
    ax3.set_xlabel(f'PC1 ({explained_variance[0]*100:.1f}%)')
    ax3.set_ylabel(f'PC2 ({explained_variance[1]*100:.1f}%)')
    ax3.grid(True, alpha=0.3)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax3)
    cbar.set_label('Time (Recent → Older)', rotation=270, labelpad=15)
    
    # 4. Principal component time series
    ax4 = fig.add_subplot(gs[1, 1:])
    
    for i, (pc, color) in enumerate(zip(['PC1', 'PC2', 'PC3'], pc_colors)):
        if pc in scores.columns:
            ax4.plot(scores.index, scores[pc], 
                    color=color, linewidth=1.5, alpha=0.8,
                    label=f'{pc} ({explained_variance[i]*100:.1f}%)')
    
    ax4.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax4.set_title('Principal Component Scores Over Time', fontsize=12, weight='bold')
    ax4.set_xlabel('Trading Date')
    ax4.set_ylabel('PC Score')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Component interpretation table
    ax5 = fig.add_subplot(gs[2, :])
    ax5.axis('off')
    
    # Interpret the first three components
    interpretations = []
    
    # PC1 interpretation (usually level/parallel shifts)
    pc1_loadings = components['PC1']
    if (pc1_loadings > 0).all() or (pc1_loadings < 0).all():
        pc1_interp = "Level/Parallel Shift"
    else:
        pc1_interp = "Mixed Movement"
    
    # PC2 interpretation (usually slope/steepness)
    pc2_loadings = components['PC2']
    if pc2_loadings.iloc[0] * pc2_loadings.iloc[-1] < 0:
        pc2_interp = "Slope/Steepness Change"
    else:
        pc2_interp = "Curvature Change"
    
    # PC3 interpretation (usually curvature)
    pc3_loadings = components['PC3']
    middle_idx = len(pc3_loadings) // 2
    if abs(pc3_loadings.iloc[middle_idx]) > abs(pc3_loadings.iloc[0]):
        pc3_interp = "Butterfly/Curvature"
    else:
        pc3_interp = "Wing Movement"
    
    interpretation_data = [
        ['PC1', f'{explained_variance[0]*100:.1f}%', pc1_interp, 'Broad market moves affecting all tenors'],
        ['PC2', f'{explained_variance[1]*100:.1f}%', pc2_interp, 'Changes in term structure slope'],
        ['PC3', f'{explained_variance[2]*100:.1f}%', pc3_interp, 'Local curve shape adjustments']
    ]
    
    # Create interpretation table
    table = ax5.table(cellText=interpretation_data,
                     colLabels=['Component', 'Variance', 'Type', 'Market Interpretation'],
                     cellLoc='left',
                     loc='center',
                     colWidths=[0.15, 0.15, 0.25, 0.45])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2.5)
    
    # Style the table
    for i in range(len(interpretation_data) + 1):
        for j in range(4):
            cell = table[(i, j)]
            if i == 0:  # Header
                cell.set_facecolor('#2c3e50')
                cell.set_text_props(weight='bold', color='white')
            else:
                cell.set_facecolor('#ecf0f1' if i % 2 == 0 else 'white')
    
    ax5.set_title('Principal Component Interpretation', fontsize=14, weight='bold', pad=20)
    
    plt.suptitle('Principal Component Analysis of Copper Futures Term Structure', 
                 fontsize=16, weight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\nPCA Summary:")
    print(f"First 3 components explain {cumulative_var[2]*100:.1f}% of variance")
    print(f"PC1 (Level): {explained_variance[0]*100:.1f}%")
    print(f"PC2 (Slope): {explained_variance[1]*100:.1f}%")
    print(f"PC3 (Curvature): {explained_variance[2]*100:.1f}%")

else:
    print("Insufficient data for PCA analysis.")

## 5. Market Regime Analysis

In [ ]:
# Market regime identification using multiple indicators
def identify_market_regimes(price_data, vol_data, liquidity_data):
    """Identify different market regimes using multiple indicators"""
    regimes = pd.DataFrame(index=price_data.index)
    
    # Front month price for trend analysis
    if 1 in price_data.columns:
        front_price = price_data[1]
        
        # Price trend (20-day moving average)
        ma_20 = front_price.rolling(20).mean()
        ma_60 = front_price.rolling(60).mean()
        
        regimes['Price_Trend'] = np.where(ma_20 > ma_60, 'Uptrend', 'Downtrend')
        
        # Price momentum
        returns_20d = front_price.pct_change(20)
        regimes['Momentum'] = pd.cut(returns_20d, 
                                   bins=[-np.inf, -0.1, -0.05, 0.05, 0.1, np.inf],
                                   labels=['Strong Down', 'Weak Down', 'Sideways', 'Weak Up', 'Strong Up'])
    
    # Volatility regime
    if 1 in vol_data.columns:
        vol_20d = vol_data[('20d_vol', 1)] if ('20d_vol', 1) in vol_data.columns else vol_data[1]
        vol_percentiles = vol_20d.rolling(252).rank(pct=True)
        
        regimes['Vol_Regime'] = pd.cut(vol_percentiles,
                                     bins=[0, 0.25, 0.75, 1],
                                     labels=['Low Vol', 'Normal Vol', 'High Vol'])
    
    # Term structure regime
    if 1 in price_data.columns and 3 in price_data.columns:
        spread_1_3 = price_data[3] - price_data[1]
        spread_percentiles = spread_1_3.rolling(252).rank(pct=True)
        
        regimes['TS_Regime'] = pd.cut(spread_percentiles,
                                    bins=[0, 0.3, 0.7, 1],
                                    labels=['Backwardation', 'Neutral', 'Contango'])
    
    # Liquidity regime
    if 'Total_Volume' in liquidity_data.columns:
        vol_ma = liquidity_data['Total_Volume'].rolling(20).mean()
        vol_percentiles = vol_ma.rolling(252).rank(pct=True)
        
        regimes['Liquidity_Regime'] = pd.cut(vol_percentiles,
                                           bins=[0, 0.33, 0.67, 1],
                                           labels=['Low Liquidity', 'Normal Liquidity', 'High Liquidity'])
    
    return regimes

# Identify regimes
regimes = identify_market_regimes(price_pivot, vol_metrics, liquidity_metrics)

# Create regime analysis visualization
fig = plt.figure(figsize=(18, 14))
gs = GridSpec(4, 2, figure=fig, hspace=0.4, wspace=0.3)

# 1. Price with regime overlay
ax1 = fig.add_subplot(gs[0, :])
if 1 in price_pivot.columns:
    front_price = price_pivot[1].dropna()
    ax1.plot(front_price.index, front_price, color='black', linewidth=1, alpha=0.7)
    
    # Color background by volatility regime
    if 'Vol_Regime' in regimes.columns:
        vol_regime_clean = regimes['Vol_Regime'].dropna()
        
        for regime, color in zip(['Low Vol', 'Normal Vol', 'High Vol'], 
                               ['lightgreen', 'lightblue', 'lightcoral']):
            regime_periods = vol_regime_clean == regime
            if regime_periods.any():
                for start, end in get_regime_periods(regime_periods):
                    ax1.axvspan(start, end, alpha=0.3, color=color, label=regime if start == get_regime_periods(regime_periods)[0][0] else "")
    
    ax1.set_title('Front Month Price with Volatility Regime Overlay', fontsize=14, weight='bold')
    ax1.set_ylabel('Price (USD/tonne)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

def get_regime_periods(regime_boolean):
    """Helper function to get start/end dates of regime periods"""
    periods = []
    start = None
    
    for date, is_regime in regime_boolean.items():
        if is_regime and start is None:
            start = date
        elif not is_regime and start is not None:
            periods.append((start, date))
            start = None
    
    if start is not None:
        periods.append((start, regime_boolean.index[-1]))
    
    return periods

# 2. Regime distribution charts
regime_columns = ['Price_Trend', 'Vol_Regime', 'TS_Regime', 'Liquidity_Regime']
colors_list = [performance_colors, risk_colors, time_colors[:3], risk_colors]

for i, (col, colors) in enumerate(zip(regime_columns, colors_list)):
    if col in regimes.columns:
        ax = fig.add_subplot(gs[1 + i//2, i%2])
        
        regime_counts = regimes[col].value_counts()
        
        wedges, texts, autotexts = ax.pie(regime_counts.values, 
                                         labels=regime_counts.index,
                                         colors=colors[:len(regime_counts)],
                                         autopct='%1.1f%%',
                                         startangle=90)
        
        ax.set_title(f'{col.replace("_", " ")} Distribution', fontsize=12, weight='bold')

# 3. Regime transition matrix (for volatility regimes)
if 'Vol_Regime' in regimes.columns:
    ax_matrix = fig.add_subplot(gs[3, :])
    
    # Calculate transition matrix
    vol_regime_shifted = regimes['Vol_Regime'].shift(1)
    transition_data = pd.crosstab(vol_regime_shifted, regimes['Vol_Regime'], normalize='index') * 100
    
    if not transition_data.empty:
        sns.heatmap(transition_data, annot=True, fmt='.1f', cmap='Blues',
                   ax=ax_matrix, cbar_kws={'label': 'Transition Probability (%)'})
        
        ax_matrix.set_title('Volatility Regime Transition Matrix', fontsize=14, weight='bold')
        ax_matrix.set_xlabel('To Regime')
        ax_matrix.set_ylabel('From Regime')

plt.suptitle('Copper Futures Market Regime Analysis', fontsize=16, weight='bold', y=0.98)
plt.tight_layout()
plt.show()

# Regime statistics
print("\nMarket Regime Analysis Summary:")
for col in regime_columns:
    if col in regimes.columns:
        current_regime = regimes[col].dropna().iloc[-1] if not regimes[col].dropna().empty else "Unknown"
        print(f"{col.replace('_', ' ')}: Currently in '{current_regime}' regime")

# Close database connection
conn.close()
print("\nDatabase connection closed. Analysis complete.")